<a href="https://colab.research.google.com/github/krooner/til/blob/main/replicate_LMRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# !pip list | grep kaggle
# !pip list | grep transformers
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

## Data Preparation
Kaggle

In [ ]:
!mkdir /root/.config/kaggle
!cp '/gdrive/MyDrive/datastore/kaggle.json' /root/.config/kaggle/kaggle.json

In [ ]:
!kaggle datasets download carrie1/ecommerce-data && unzip ecommerce-data.zip

Dataset URL: https://www.kaggle.com/datasets/carrie1/ecommerce-data
License(s): unknown
Archive:  ecommerce-data.zip
  inflating: data.csv                


In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv("/content/data.csv", encoding='latin-1')
display(dataframe.shape)
display(dataframe.head())

(541909, 8)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
dataframe = dataframe.loc[~dataframe.CustomerID.isnull()].astype({'CustomerID': int})

display(dataframe.shape)
display(dataframe.head())

(406829, 8)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [ ]:
# dataframe.CustomerID.unique().shape # 4372
user_df = dataframe.sort_values('InvoiceDate')
user_df = dataframe.groupby('CustomerID').agg({'Description': lambda x: "->".join(x), 'Quantity': len}).reset_index()
user_df = user_df.loc[user_df.Quantity>1]

user_df['user_sequence'] = user_df.Description.apply(lambda x: "->".join(x.split("->")[:-1]))
user_df['user_item'] = user_df.Description.apply(lambda x: x.split("->")[-1])
user_df.head()
# user_df.CustomerID.unique().shape # 4293

,CustomerID,Description,Quantity,user_sequence,user_item
0,12346,MEDIUM CERAMIC TOP STORAGE JAR->MEDIUM CERAMIC...,2,MEDIUM CERAMIC TOP STORAGE JAR,MEDIUM CERAMIC TOP STORAGE JAR
1,12347,BLACK CANDELABRA T-LIGHT HOLDER->AIRLINE BAG V...,182,BLACK CANDELABRA T-LIGHT HOLDER->AIRLINE BAG V...,MINI PLAYING CARDS DOLLY GIRL
2,12348,72 SWEETHEART FAIRY CAKE CASES->60 CAKE CASES ...,31,72 SWEETHEART FAIRY CAKE CASES->60 CAKE CASES ...,POSTAGE
3,12349,PARISIENNE CURIO CABINET->SWEETHEART WALL TIDY...,73,PARISIENNE CURIO CABINET->SWEETHEART WALL TIDY...,POSTAGE
4,12350,CHOCOLATE THIS WAY METAL SIGN->METAL SIGN NEIG...,17,CHOCOLATE THIS WAY METAL SIGN->METAL SIGN NEIG...,BLUE POLKADOT PASSPORT COVER


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(user_df)

In [ ]:
# from transformers import AutoTokenizer
from transformers import GPT2TokenizerFast
model_tag = 'skt/kogpt2-base-v2'

# tokenizer = AutoTokenizer.from_pretrained(model_tag)
tokenizer = GPT2TokenizerFast.from_pretrained(model_tag)

In [ ]:
encoded = tokenizer.encode('<s></s>')
print(encoded)
print(tokenizer.convert_ids_to_tokens(encoded))
decoded = tokenizer.decode(encoded)
print(decoded)

[0, 1]
['<s>', '</s>']
<s></s>


In [ ]:
encoded = tokenizer.encode('<|endoftext|><|endoftext|>')
print(encoded)
print(tokenizer.convert_ids_to_tokens(encoded))
decoded = tokenizer.decode(encoded)
print(decoded)

[51200, 51200]
['<|endoftext|>', '<|endoftext|>']
<|endoftext|><|endoftext|>


In [ ]:
from tokenizers.processors import TemplateProcessing

tokenizer.bos_token = '<s>'
tokenizer.bos_token_id = 0

tokenizer.eos_token = '</s>'
tokenizer.eos_token_id = 1

tokenizer.pad_token = '<pad>'
tokenizer.pad_token_id = 3

tokenizer._tokenizer.post_processor = TemplateProcessing(
    single="<s> $0 </s>",
    special_tokens=[("<s>", tokenizer.bos_token_id), ("</s>", tokenizer.eos_token_id)]
)

In [ ]:
def tokenize_data(inputs):
  tokenized_data = tokenizer(inputs['Description'], padding=True, truncation=True, max_length=32)
  return tokenized_data

dataset_mapped = dataset.map(
    tokenize_data,
    batched=True
)

Map:   0%|          | 0/4293 [00:00<?, ? examples/s]

In [ ]:
dataset_mapped

Dataset({
    features: ['CustomerID', 'Description', 'Quantity', 'user_sequence', 'user_item', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 4293
})

In [ ]:
tokenizer.convert_ids_to_tokens(range(10))

['<s>',
 '</s>',
 '<usr>',
 '<pad>',
 '<sys>',
 '<unk>',
 '<mask>',
 '<d>',
 '</d>',
 '<unused0>']

In [ ]:
# sample_input_ids = dataset_mapped[0]['input_ids']
sample_texts = [
    "Hello World",
    "We are the world"
]
tokenize_text = tokenizer(sample_texts, padding=True, truncation=True, max_length=16, return_tensors='pt')

eos_token_pos = (tokenize_text['input_ids'] == tokenizer.eos_token_id).nonzero()
display(tokenize_text['input_ids'])
display(tokenize_text['input_ids'] == tokenizer.eos_token_id)
display(eos_token_pos[:, -1])

tensor([[    0, 10553, 14308,   453, 30194,     1,     3,     3],
        [    0, 36383,   739, 27797, 11370, 18896, 25228,     1]])

tensor([[False, False, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False,  True]])

tensor([5, 7])

In [ ]:
display(tokenizer.bos_token, tokenizer.bos_token_id)
display(tokenizer.eos_token, tokenizer.eos_token_id)

'<s>'

0

'</s>'

1

## Modeling

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
# from transformers import AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_tag)
# tokenizer = AutoTokenizer.from_pretrained(model_tag)

display(model)
# display(tokenizer)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

GPT2Config {
  "_attn_implementation_autoset": true,
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
   

In [ ]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

from tokenizers.processors import TemplateProcessing

class LMRecModel(nn.Module):
  def __init__(self, model_name="skt/kogpt2-base-v2"):
    super().__init__()

    self.language_model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True)
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.n_dim = self.language_model.config.n_embd

    self.coef = 1e-3

    self.user_linear = nn.Linear(self.n_dim, self.n_dim)
    self.item_linear = nn.Linear(self.n_dim, self.n_dim)
    self.sigmoid = nn.Sigmoid()

    self.loss_fct = nn.BCEWithLogitsLoss()

    self.tokenizer.bos_token = '<s>'
    self.tokenizer.bos_token_id = 0

    self.tokenizer.eos_token = '</s>'
    self.tokenizer.eos_token_id = 1

    self.tokenizer.pad_token = '<pad>'
    self.tokenizer.pad_token_id = 3

    self.tokenizer._tokenizer.post_processor = TemplateProcessing(
        single="<s> $0 </s>",
        special_tokens=[("<s>", self.tokenizer.bos_token_id), ("</s>", self.tokenizer.eos_token_id)]
    )

  def forward(self, user_input_ids, user_attention_mask, item_input_ids, item_attention_mask, labels=None):
    user_outputs = self.language_model(
        input_ids=user_input_ids,
        attention_mask=user_attention_mask,
        labels=user_input_ids.clone()
    )
    causal_lm_loss = user_outputs.loss

    user_eos_indices = (user_input_ids == self.tokenizer.eos_token_id).nonzero(as_tuple=True)[-1]
    item_eos_indices = (item_input_ids == self.tokenizer.eos_token_id).nonzero(as_tuple=True)[-1]

    last_hidden_state_user = user_outputs.hidden_states[-1]
    # user_embedding = user_outputs.hidden_states[-1][:, -1, :] # (batch_size, max_length, n_dim) -> (batch_size, n_dim)

    # user_embedding = torch.Tensor(
    #     [last_hidden_state_user[i, eos_indices[i], :] for i in range(len(user_input_ids))]
    # ).reshape((-1, self.n_dim)) # (batch_size, max_length, n_dim) -> (batch_size, n_dim)

    user_embedding = last_hidden_state_user[torch.arange(len(user_input_ids)), user_eos_indices]

    with torch.no_grad():
      item_outputs = self.language_model(
          input_ids=item_input_ids,
          attention_mask=item_attention_mask,
      )

    last_hidden_state_item = item_outputs.hidden_states[-1]
    # item_embedding = item_outputs.hidden_states[-1][:, -1, :] # (batch_size, max_length, n_dim) -> (batch_size, n_dim)

    # item_embedding = torch.Tensor(
    #     [last_hidden_state_item[i, eos_indices[i], :] for i in range(len(item_input_ids))]
    # ).reshape((-1, self.n_dim))

    item_embedding = last_hidden_state_item[torch.arange(len(item_input_ids)), item_eos_indices]

    user_hidden = self.user_linear(user_embedding)
    item_hidden = self.item_linear(item_embedding)

    user_item_dot_product = (user_hidden * item_hidden).sum(dim=1)

    user_item_prob = self.sigmoid(user_item_dot_product)

    rec_loss = self.loss_fct(user_item_prob, labels.float())

    total_loss = causal_lm_loss + self.coef * rec_loss

    return total_loss

  # TODO
  def encode_user(self, user_input_ids, user_attention_mask):
    self.language_model.eval()
    with torch.no_grad():
      user_outputs = self.language_model(
          input_ids=user_input_ids,
          attention_mask=user_attention_mask
      )

    user_eos_indices = (user_input_ids == self.tokenizer.eos_token_id).nonzero(as_tuple=True)[-1]

    last_hidden_state_user = user_outputs.hidden_states[-1].cpu().detach()

    user_embedding = last_hidden_state_user[torch.arange(len(user_input_ids)), user_eos_indices]

    user_hidden = self.user_linear(user_embedding)

    return user_hidden

  # TODO
  def encode_item(self, item_input_ids, item_attention_mask):
    self.language_model.eval()
    with torch.no_grad():
      item_outputs = self.language_model(
          input_ids=item_input_ids,
          attention_mask=item_attention_mask
      )

    item_eos_indices = (item_input_ids == self.tokenizer.eos_token_id).nonzero(as_tuple=True)[-1]

    last_hidden_state_item = item_outputs.hidden_states[-1].cpu().detach()

    item_embedding = last_hidden_state_item[torch.arange(len(item_input_ids)), item_eos_indices]

    item_hidden = self.item_linear(user_embedding)

    return item_hidden

  def get_user_item_probability(self, user_hidden, item_hidden):

    return self.sigmoid(torch.matmul(user_hidden, item_hidden.t()))

    # user_item_dot_product = (user_hidden * item_hidden).sum(dim=1)
    # user_item_prob = self.sigmoid(user_item_dot_product)
    # return user_item_prob.detach().numpy()

In [ ]:
model = LMRecModel()

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [ ]:
user_seq = ["안녕하세요. 고객센터입니다.", "Hello World"]
item_seq = ["세탁기 소음 문제 해결 방법 안내", "How are you"]

user_inputs = tokenizer(user_seq, padding=True, truncation=True, max_length=32, return_tensors='pt')
item_inputs = tokenizer(item_seq, padding=True, truncation=True, max_length=32, return_tensors='pt')

displays

lm_labels = user_inputs["input_ids"].clone()
lm_labels

tensor([[    0, 25906,  8702,  7801, 25856, 12298, 12092, 21154,     1],
        [    0, 10553, 14308,   453, 30194,     1,     3,     3,     3]])

In [ ]:
lm_labels[lm_labels==tokenizer.pad_token_id] = -100
display(lm_labels)

labels = torch.tensor([1, 0])
display(labels)

tensor([[    0, 25906,  8702,  7801, 25856, 12298, 12092, 21154,     1],
        [    0, 10553, 14308,   453, 30194,     1,  -100,  -100,  -100]])

tensor([1, 0])

In [ ]:
outputs = model(
    user_input_ids=user_inputs['input_ids'],
    user_attention_mask=user_inputs['attention_mask'],
    item_input_ids=item_inputs['input_ids'],
    item_attention_mask=item_inputs['attention_mask'],
    labels=labels,
  )
outputs

tensor(8.9619, grad_fn=<AddBackward0>)

In [ ]:
user_embedding = model.encode_user(
    user_input_ids=user_inputs['input_ids'],
    user_attention_mask=user_inputs['attention_mask'],
  )

item_embedding = model.encode_item(
    item_input_ids=item_inputs['input_ids'],
    item_attention_mask=item_inputs['attention_mask'],
  )
user_embedding.shape, item_embedding.shape

(torch.Size([2, 768]), torch.Size([2, 768]))

In [ ]:
user_item_prob = model.get_user_item_probability(user_embedding, item_embedding)
user_item_prob

tensor([[0.3622, 0.0004],
        [0.0561, 0.0991]], grad_fn=<SigmoidBackward0>)